In [ ]:
#hide
from nbdev.showdoc import *

# recocido_simulado

> To run recocido simulado.

In [ ]:
default_exp core

In [ ]:
import random
import copy
import math

In [ ]:
def toint(n):
    n = int(n)
    return n

def intolist(inp):
    inp = inp.split()
    inp = list(map(toint, inp))
    return inp

def inpt():
    N = int(input())
    T = int(input())
    dist = []
    for i in range(N-1):
        inp = input()
        dist.append(intolist(inp))
    return N, T, dist

In [ ]:
def fillmatrix(distances, N):
    distances_complete = []
    rows = cols = N
    distances_complete = [([0]*cols) for i in range(rows)]
    for i in range(len(distances)):
        for j in range(len(distances[i])):
            distances_complete[i][j+1+i] = distances[i][j]
            distances_complete[j+1+i][i] = distances[i][j]

    return distances_complete

In [ ]:
def greedy(distances_complete, N):
    greedy_sol = [0]
    c_actual = 0
    c_obj = 1
    mini = distances_complete[c_actual][1]
    for i in range(N-2):
        for j in range(N):
            if distances_complete[c_actual][j] < mini and j not in greedy_sol and distances_complete[c_actual][j] != 0:
                mini = distances_complete[c_actual][j]
                c_obj = j
        greedy_sol.append(c_obj)
        c_actual = c_obj
        mini = 100000000
    missing = list(set(range(0, N)) - set(greedy_sol))
    greedy_sol.append(missing[0])

    return greedy_sol

In [ ]:
def evaluate(distances_complete, solution):
    cost = 0
    for i in range(len(solution)-1):
        cost += distances_complete[solution[i]][solution[i+1]]
    cost += distances_complete[solution[0]][solution[-1]]
    
    return cost

def new_temp(T):
    return T*0.85

In [ ]:
def recocido(N, T, distances_complete):
    best_sol = greedy(distances_complete, N)
    best_sol_cost = evaluate(distances_complete, best_sol)
    overall_best = best_sol
    overall_best_cost = best_sol_cost
    while T > 0.0001:
        pos1 = random.randint(0, N-1)
        while True:
            pos2 = random.randint(0, N-1)
            if pos2 != pos1:
                break
        contender = copy.deepcopy(best_sol)
        contender[pos1], contender[pos2] = contender[pos2], contender[pos1]
        contender_cost = evaluate(distances_complete, contender)
        if contender_cost < best_sol_cost:
            best_sol_cost = contender_cost
            best_sol = contender
            if best_sol_cost < overall_best_cost:
                overall_best_cost = best_sol_cost
                overall_best = best_sol
        else:
            if random.random() < math.exp(-(contender_cost - best_sol_cost)/T):
                best_sol_cost = contender_cost
                best_sol = contender
                if best_sol_cost < overall_best_cost:
                    overall_best_cost = best_sol_cost
                    overall_best = best_sol

        T = new_temp(T)
    
    return overall_best, overall_best_cost